In [67]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
# from api.src.schemas.cataloguing.edit import BfEdit
from pydantic import BaseModel
from typing import Union

In [68]:
class Data(BaseModel):
    action: str
    bf: str
    value: Union[dict, str]

class BfEdit(BaseModel):
    listData: list[Data]

In [30]:
au_update = FusekiUpdate('http://localhost:3030', 'acervo')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [121]:
obj = {
  "listData": [
    {
      "action": "edit",
      "bf": "title",
      "value": {
          "mainTitle": "The journey of humanity v.4"
      } 
    },
    {
      "action": "add",
      "bf": "type",
      "value": "Monograph"
    },
    {
      "action": "edit",
      "bf": "classification",
      "value":{
          "classificationPortion": "338.9001",
          "itemPortion": "O48j"
      } 
    },
    {
      "action": "add",
      "bf": "content",
      "value":{
          "uri": "http://id.loc.gov/vocabulary/contentTypes/txt",
          "label": "Texto"
      } 
    },
    {
      "action": "remove",
      "bf": "contribution",
      "value":{
          "type": ["bflc:PrimaryContribution" , "bf:Contribution" ],
          "agent": "http://id.loc.gov/rwo/agents/n86870523",
          "label": "AUTHOR",
          "role": "http://id.loc.gov/vocabulary/relators/TESTE"
      } 
    }
  ]
}

request = BfEdit(**obj)
request

BfEdit(listData=[Data(action='edit', bf='title', value={'mainTitle': 'The journey of humanity v.4'}), Data(action='add', bf='type', value='Monograph'), Data(action='edit', bf='classification', value={'classificationPortion': '338.9001', 'itemPortion': 'O48j'}), Data(action='add', bf='content', value={'uri': 'http://id.loc.gov/vocabulary/contentTypes/txt', 'label': 'Texto'}), Data(action='remove', bf='contribution', value={'type': ['bflc:PrimaryContribution', 'bf:Contribution'], 'agent': 'http://id.loc.gov/rwo/agents/n86870523', 'label': 'AUTHOR', 'role': 'http://id.loc.gov/vocabulary/relators/TESTE'})])

In [120]:
uri = 'https://bibliokeia.com/resources/work/bk-1'
acervo = FusekiUpdate('http://localhost:3030', 'acervo')

def EditTitle(uri, data):
    up = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        WITH <{uri}>
        DELETE {{  <{uri}> bf:title ?o .
                        ?o ?p ?s}}
        INSERT {{ <{uri}> bf:title ?o .
                    ?o rdf:type bf:Title .
                    ?o bf:mainTitle "{data.value.get('mainTitle')}" .
                    { f'?o bf:subtitle "{data.value.get("subtitle")}"' if data.value.get('subtitle') else '' }
                      }}
        WHERE {{ <{uri}> bf:title ?o .
                        ?o ?p ?s }} """
    # print(up)
    response = acervo.run_sparql(up)
    print(response.convert())

def EditType(uri, data):

    if data.action == 'remove':
        sparql = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                DELETE DATA
                {{ GRAPH <{uri}>
                {{ <{uri}> rdf:type  bf:{data.value} }} }} ;"""
        response = acervo.run_sparql(sparql)
        print(response.convert())
    if data.action == 'add':
        sparql = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                INSERT DATA
                {{ GRAPH <{uri}>
                {{ <{uri}> rdf:type  bf:{data.value} }} }} ;"""
        response = acervo.run_sparql(sparql)
        print(response.convert())

def EditClassification(uri, data):
    sparql = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        WITH <{uri}>
        DELETE {{  <{uri}> bf:classification ?o .
                        ?o ?p ?s}}
        INSERT {{ <{uri}> bf:classification ?o .
                    ?o rdf:type bf:ClassificationDdc .
                    ?o bf:classificationPortion "{data.value.get('classificationPortion')}" .
                    { f'?o bf:itemPortion "{data.value.get("itemPortion")}"' if data.value.get('itemPortion') else '' }
                    }}
        WHERE {{ <{uri}> bf:classification ?o .
                        ?o ?p ?s }} """
    # print(up)
    response = acervo.run_sparql(sparql)
    print(response.convert())
    
def EditContent(uri, data):

    if data.action == 'remove':
        sparql = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                DELETE DATA
                {{ GRAPH <{uri}>
                {{ <{uri}> bf:content <{data.value.get('uri')}> }} }} ; """
        response = acervo.run_sparql(sparql)
        print(response.convert())
    if data.action == 'add':
        sparql = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                INSERT DATA
                {{ GRAPH <{uri}>
                {{ <{uri}> bf:content <{data.value.get('uri')}> }} }} ; """
        response = acervo.run_sparql(sparql)
        print(response.convert())

def EditContribution(uri, data):

    if data.action == 'edit':
        sparql = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
        WITH <{uri}>
        DELETE {{  <{uri}> bf:contribution ?o .
                        ?o ?p ?s }}
        INSERT {{ <{uri}> bf:contribution ?o .
                  ?o rdf:type {", ".join(data.value.get('type'))} .
                  ?o bf:agent  <{data.value.get('agent')}> .
                  { f'?o bf:role "{data.value.get("role")}"' if data.value.get('role') else '' }
                }}
        WHERE {{ <{uri}> bf:contribution ?o .
                    ?o bf:agent  <{data.value.get('agent')}> .}} """
        response = acervo.run_sparql(sparql)
        print(response.convert())
    elif data.action == 'remove':
        sparql = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
        WITH <{uri}>
        DELETE {{ <{uri}> bf:contribution ?o .
                    ?o ?p ?s }}
        WHERE {{ <{uri}> bf:contribution ?o .
                    ?o bf:agent  <{data.value.get('agent')}> . }} """
        response = acervo.run_sparql(sparql)
        print(response.convert())
    elif data.action == 'add':
        
        
        



In [119]:
sparql = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
        WITH <{uri}>
        DELETE {{  <{uri}> bf:contribution ?o .
                        ?o ?p ?s }}
        INSERT {{ <{uri}> bf:contribution ?o .
                  ?o rdf:type {", ".join(data.value.get('type'))} .
                  ?o bf:agent  <{data.value.get('agent')}> .
                  { f'?o bf:role "{data.value.get("role")}"' if data.value.get('role') else '' }
                }}
        WHERE {{ <{uri}> bf:contribution ?o .
                        ?o ?p ?s }} """
print(sparql)

PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
        WITH <https://bibliokeia.com/resources/work/bk-1>
        DELETE {  <https://bibliokeia.com/resources/work/bk-1> bf:contribution ?o .
                        ?o ?p ?s }
        INSERT { <https://bibliokeia.com/resources/work/bk-1> bf:contribution ?o .
                  ?o rdf:type bflc:PrimaryContribution, bf:Contribution .
                  ?o bf:agent  <http://id.loc.gov/rwo/agents/n86870523> .
                  ?o bf:role "http://id.loc.gov/vocabulary/relators/TESTE"
                }
        WHERE { <https://bibliokeia.com/resources/work/bk-1> bf:contribution ?o .
                        ?o ?p ?s } 


In [116]:
sparql = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
                PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
                INSERT DATA
                {{ GRAPH <{uri}>
                {{ <{uri}> bf:contribution [ rdf:type  bflc:PrimaryContribution , bf:Contribution ;
                                   bf:agent  <http://id.loc.gov/rwo/agents/n86870523> ;
                                   bf:role   <http://id.loc.gov/vocabulary/relators/aut>
                                 ] }} }} ; """
response = acervo.run_sparql(sparql)
print(response.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [117]:
sparql = f"""PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
        WITH <{uri}>
        DELETE {{  <{uri}> bf:contribution ?o .
                        ?o ?p ?s }}
        INSERT {{ <{uri}> bf:contribution ?o .
                  ?o rdf:type bflc:PrimaryContribution , bf:Contribution .
                  ?o bf:agent  <http://id.loc.gov/rwo/agents/n86870523> .
                  ?o bf:role   <http://id.loc.gov/vocabulary/relators/TESTE> }}
        WHERE {{ <{uri}> bf:contribution ?o .
                        ?o ?p ?s }} """
response = acervo.run_sparql(sparql)
print(response.convert())


{'statusCode': 200, 'message': 'Update succeeded'}


In [125]:
sparql = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX bflc:    <http://id.loc.gov/ontologies/bflc/> 
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            INSERT DATA
            {{ GRAPH <{uri}>
            {{ <{uri}> bf:contribution [ a bflc:PrimaryContribution,
                            bf:Contribution ;
                        bf:agent <http://id.loc.gov/rwo/agents/n86870523> ; 
                        bf:role <http://id.loc.gov/vocabulary/relators/aut1> ] ; }} }} ; """
response = acervo.run_sparql(sparql)
print(response.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [122]:
for data in request.listData:
    if data.bf == 'title':
        # EditTitle(uri, data)
            # print(data.value.get('subtitle'))
        pass
    elif data.bf == 'type':
        # EditTitle(uri, data)
        # print(data)
        pass
    elif data.bf == 'classification':
        # EditClassification(uri, data)
        # print(data)
        pass
    elif data.bf == 'content':
        # EditContent(uri, data)
        # print(data)
        pass
    elif data.bf == 'contribution':
        EditContribution(uri, data)
        # print(data)


{'statusCode': 200, 'message': 'Update succeeded'}


In [3]:


solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)


'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">177</int>\n</lst>\n</response>\n'

In [2]:
au_update = FusekiUpdate('http://localhost:3030', 'acervo')

In [3]:
clear = "CLEAR DEFAULT"
r = au_update.run_sparql(clear)
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [6]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [10]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/subject/', headers=headers, json=r.json())
response.json()

{'id': 'bkau-2', 'jena': 'Update succeeded'}